In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 7.8 MB/s 
     |████████████████████████████████| 181 kB 55.1 MB/s 
     |████████████████████████████████| 157 kB 65.8 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 157 kB 66.8 MB/s 
     |████████████████████████████████| 157 kB 69.8 MB/s 
     |████████████████████████████████| 157 kB 75.5 MB/s 
     |████████████████████████████████| 157 kB 73.1 MB/s 
     |████████████████████████████████| 157 kB 71.4 MB/s 
     |████████████████████████████████| 157 kB 62.3 MB/s 
     |████████████████████████████████| 156 kB 77.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=399af2f548d32b6bf94032640a941558e0601d9037369fbedc6b97b0f29fd3d0
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Succ

In [2]:
%cd drive/MyDrive/data/aT/scaled_data/

/content/drive/MyDrive/data/aT/scaled_data


In [3]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import wandb
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob import glob
from sklearn.model_selection import train_test_split
import random
import os

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
torch.random.manual_seed(2020320120)
random.seed(2020320120)
np.random.seed(2020320120)

In [5]:
data_list = glob('.//train/*.csv')
epoch = 100
batch = 15
tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

In [6]:
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(len(df) - seqence_length):
        tmp = df[index: index + seqence_length].values
        tmp = np.vstack(tmp).astype(np.float)
        tmp = torch.from_numpy(tmp)
        result.append(tmp)

    return np.array(result)


def make_dataset(i):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    for j in df.columns:
        df[j] = df[j].replace({' ': np.nan})

    # 사용할 열 선택 및 index 설정
    df.drop(tr_del_list, axis=1, inplace=True)
    df.set_index('datadate', drop=True, inplace=True)

    # nan 처리
    df = df.fillna(0)

    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y

    return xdata, ydata


class windowDataset(Dataset):
    def __init__(self, data):
        self.xdata, self.ydata = make_dataset(data)

    def __len__(self):
        return len(self.xdata)

    def __getitem__(self, idx):
        return self.xdata[idx], self.ydata[idx].reshape(-1)

In [9]:
class Flag:
    def __init__(self, flags):
        for key, value in flags.items():
            if isinstance(value, dict):
                self.__dict__[key] = Flag(value)
            else:
                self.__dict__[key] = value

In [15]:
def train(flags):
    name = f'experiments_dropout_{flags["data_num"]:03d}'
    flags['data_path'] = f'./train/train_{flags["data_num"]}.csv'

    wandb.init(
        project="Nong_Linear", 
        entity="deep-overflow", 
        config=flags,
        name=name
    )

    # Flag # ====================
    # 다양한 Regularization 시도해보기
    # ===========================
    flags = Flag(flags)

    # Dataset # ====================
    train_dataset = windowDataset(flags.data_path)

    # DataLoader # ====================
    train_dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=flags.batch_size,
        shuffle=True
    )

    # Net # ====================
    # Dropout
    # BatchNorm
    # ==========================
    net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(14 * 49, flags.model.hid_dim),
        nn.Dropout(flags.dropout),
        nn.LeakyReLU(),
    )

    for _ in range(flags.model.nlayers - 2):
        net.append(nn.Linear(flags.model.hid_dim, flags.model.hid_dim))
        net.append(nn.Dropout(flags.dropout))
        net.append(nn.LeakyReLU())
    
    net.append(nn.Linear(flags.model.hid_dim, 28))
    net.append(nn.Dropout(flags.dropout))
    net.append(nn.ReLU())

    # Criterion # ====================
    # 다양한 로스 시도해보기
    # ================================
    criterion = nn.L1Loss()

    # Optimizer # ====================
    # 다양한 옵티마이저 시도해보기
    # ================================
    optimizer = optim.Adam(
        params=net.parameters(), 
        lr=flags.lr,
        betas=(0.9, 0.999),
    )

    # Device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    net = net.to(device)

    # Train
    for epoch in range(flags.epochs):
            
        epoch_train_loss = 0.0

        for inputs, labels in train_dataloader:
            inputs, labels = inputs.float().to(device), labels.to(device)

            with torch.set_grad_enabled(True):
                outputs = net(inputs)
                loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss.item()
        
        wandb.log({'loss': epoch_train_loss / len(train_dataloader)})
        if (epoch + 1) % 100 == 0:
            print(f'[epoch : {epoch + 1} / {flags.epochs}] Train Loss : {epoch_train_loss / len(train_dataloader)}')

In [16]:
flags = {
    'epochs': 1000,
    'lr': 1e-3,
    'batch_size': 64,
    'data_num': 3,
    'model': {
        'nlayers': 10,
        'hid_dim': 1024,
    },
    'optim': 'Adam',
    'criterion': 'L1Loss',
}

In [17]:
for i in range(37):
    flags = {
        'epochs': 1000,
        'lr': 1e-3,
        'batch_size': 64,
        'data_num': i,
        'model': {
            'nlayers': 10,
            'hid_dim': 1024,
        },
        'optim': 'Adam',
        'criterion': 'L1Loss',
        'dropout': 0.5
    }

    train(flags)

[epoch : 100 / 1000] Train Loss : 2032.1838272758152
[epoch : 200 / 1000] Train Loss : 2005.3613971212635
[epoch : 300 / 1000] Train Loss : 1997.1155952785325
[epoch : 400 / 1000] Train Loss : 1998.3198401409647
[epoch : 500 / 1000] Train Loss : 1985.0549422554348
[epoch : 600 / 1000] Train Loss : 2006.2489066745925
[epoch : 700 / 1000] Train Loss : 2010.6656387992527
[epoch : 800 / 1000] Train Loss : 1983.4836850373642
[epoch : 900 / 1000] Train Loss : 2007.0825301460598
[epoch : 1000 / 1000] Train Loss : 1975.4445694633152


loss,█▇▄▅▄▄▃▂▃▃▂▂▂▃▃▁▂▃▂▂▂▂▂▁▂▂▂▂▂▁▃▁▁▄▂▂▁▂▂▁
loss,1975.44457


[epoch : 100 / 1000] Train Loss : 734.9507870881454
[epoch : 200 / 1000] Train Loss : 715.5313163425611
[epoch : 300 / 1000] Train Loss : 716.9336309018342
[epoch : 400 / 1000] Train Loss : 720.1256209663723
[epoch : 500 / 1000] Train Loss : 718.9062526537025
[epoch : 600 / 1000] Train Loss : 712.6665490191916
[epoch : 700 / 1000] Train Loss : 718.0658887780231
[epoch : 800 / 1000] Train Loss : 716.6471239172894
[epoch : 900 / 1000] Train Loss : 713.8553360648777
[epoch : 1000 / 1000] Train Loss : 717.8405230978261


loss,▄▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▂▂▁█▂▂▁▁▁
loss,717.84052


[epoch : 100 / 1000] Train Loss : 1158.7991571841033
[epoch : 200 / 1000] Train Loss : 1160.4326623004415
[epoch : 300 / 1000] Train Loss : 1149.9455407184103
[epoch : 400 / 1000] Train Loss : 1139.7864990234375
[epoch : 500 / 1000] Train Loss : 1133.2248641304348
[epoch : 600 / 1000] Train Loss : 1146.7863557235055
[epoch : 700 / 1000] Train Loss : 1134.5919959026835
[epoch : 800 / 1000] Train Loss : 1131.629657247792
[epoch : 900 / 1000] Train Loss : 1141.523777173913
[epoch : 1000 / 1000] Train Loss : 1140.8334270974865


...

[epoch : 100 / 1000] Train Loss : 2034.4707190472147
[epoch : 200 / 1000] Train Loss : 1995.441650390625
[epoch : 300 / 1000] Train Loss : 1985.9578326681385
[epoch : 400 / 1000] Train Loss : 1993.4496858016305
[epoch : 500 / 1000] Train Loss : 1955.5160230553668
[epoch : 600 / 1000] Train Loss : 1960.426609205163
[epoch : 700 / 1000] Train Loss : 1940.7145889945652
[epoch : 800 / 1000] Train Loss : 1951.916259765625
[epoch : 900 / 1000] Train Loss : 1945.760789954144
[epoch : 1000 / 1000] Train Loss : 1970.9895125679348


loss,██▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▁▂▂▁▂▂▂▂▂▂▁▂▂▂▁▂
loss,1970.98951


[epoch : 100 / 1000] Train Loss : 1280.3587752632473
[epoch : 200 / 1000] Train Loss : 1255.5494225543478
[epoch : 300 / 1000] Train Loss : 1255.6453485903533
[epoch : 400 / 1000] Train Loss : 1250.3984162703805
[epoch : 500 / 1000] Train Loss : 1269.1310716711957
[epoch : 600 / 1000] Train Loss : 1246.56322711447
[epoch : 700 / 1000] Train Loss : 1258.699213442595
[epoch : 800 / 1000] Train Loss : 1254.6049910835598
[epoch : 900 / 1000] Train Loss : 1254.405857252038
[epoch : 1000 / 1000] Train Loss : 1257.1908330502717


loss,█▆▃▃▃▃▃▁▃▁▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▁▁▁▂▁▂▁▁▂▂▂▂▂▂▂
loss,1257.19083


[epoch : 100 / 1000] Train Loss : 957.840308147928
[epoch : 200 / 1000] Train Loss : 957.8999925696331
[epoch : 300 / 1000] Train Loss : 955.0038903277854
[epoch : 400 / 1000] Train Loss : 946.2816109035326
[epoch : 500 / 1000] Train Loss : 955.1778405230979
[epoch : 600 / 1000] Train Loss : 945.9367728855299
[epoch : 700 / 1000] Train Loss : 957.304515009341
[epoch : 800 / 1000] Train Loss : 940.8581197987432
[epoch : 900 / 1000] Train Loss : 969.7198247494905
[epoch : 1000 / 1000] Train Loss : 945.2848457668139
